<a href="https://colab.research.google.com/github/gfdiazc/AI_Agent/blob/main/Crew_AI_An%C3%A1lisis_Competitivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Celda 1: Importación y Configuración de Dependencias

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install crewai crewai_tools -q -U pypdf chromadb google-generativeai langchain langchain-google-genai google-generativeai spider-client firecrawl-py

In [70]:
import os
import time
import logging
import google.generativeai as genai
from google.colab import userdata
from spider import Spider
from crewai import LLM, Agent, Task, Crew, Process
from crewai_tools import (
    SerperDevTool,
    ScrapeWebsiteTool,
    PDFSearchTool,
    SpiderTool,
    WebsiteSearchTool,
    FirecrawlCrawlWebsiteTool,
    FirecrawlScrapeWebsiteTool,
    FirecrawlSearchTool
)
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from openai import OpenAI
from crewai.tools import BaseTool
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from firecrawl import FirecrawlApp

# 🔑 Configurar las claves API desde Google Colab
load_dotenv()
os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_AI_KEY')  # OpenAI GPT
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_2')  # Gemini
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_KEY')  # Serper
os.environ["SPIDER_API_KEY"] = userdata.get('spider_key')  # Spider
os.environ["FIRECRAWLER_API_KEY"] = userdata.get('TU_FIRECRAWL_API_KEY')  # Firecrawler
os.environ["ANTHROPIC_API_KEY"] = userdata.get("anthropic")
os.environ["DEEPSEEK_API_KEY"] = userdata.get("DEEPSEEK_API_KEY")

firecrawl_app = FirecrawlApp(api_key=userdata.get('TU_FIRECRAWL_API_KEY'))

In [69]:
deepseek_llm = LLM(
    model="deepseek/deepseek-chat",
    api_key=os.environ["DEEPSEEK_API_KEY"],
    temperature=1.5,
)


gemini_llm = LLM(
    model="gemini/gemini-1.5-pro-latest",
    api_key=os.environ["GOOGLE_API_KEY"],
    temperature=1.5,
)

claude_llm = LLM(
    model="anthropic/claude-2",
    api_key=os.environ["ANTHROPIC_API_KEY"],
    temperature=1.5,
)

In [68]:
# 🔹 Herramienta para investigación
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
spider_tool = SpiderTool()
firecrawl_crawl_tool = FirecrawlCrawlWebsiteTool(api_key=os.environ.get('FIRECRAWLER_API_KEY'))
firecrawl_scrape_tool = FirecrawlScrapeWebsiteTool(api_key=os.environ.get('FIRECRAWLER_API_KEY'))


class FirecrawlSearchTool:
    def __init__(self, api_key, **kwargs):
        super().__init__(**kwargs)
        self.api_key = api_key
        self._initialize_firecrawl()

    def _initialize_firecrawl(self) -> None:
        try:
            # Removed the FIRECRAWL_AVAILABLE check
            # Directly initialize FirecrawlApp with the provided api_key
            self._firecrawl = FirecrawlApp(api_key=self.api_key)
        except Exception as e:
            raise ImportError(f"Could not initialize FirecrawlApp: {e}") from e


# Configurar Website tool LLM
website_search_tool = WebsiteSearchTool(
    config=dict(
        llm=dict(
            provider="google",  # Se usa Google como proveedor de LLM
            config=dict(
                model="gemini-1.5-pro-latest",  # Modelo de última generación
                temperature=0.5,
            ),
        ),
        embedder=dict(
            provider="google",  # Se usa Google para embeddings
            config=dict(
                model="models/embedding-001",  # Mejor modelo de embeddings disponible
                task_type="retrieval_document",
            ),
        ),
    )
)

# Configurar PDF tool LLM
pdf_tool_industria = PDFSearchTool(
    pdf="/content/Telco industry breakfast (Google).pdf",
    config=dict(
        llm=dict(
            provider="google", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gemini/gemini-1.5-pro-latest",
                temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google", # or openai, ollama, ...
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)


# Configurar PDF tool LLM
pdf_tool_satisfaccion = PDFSearchTool(
    pdf="/content/19221.24Q3 Satisfacción 2024 - Presentación v3.pptx.pdf",
    config=dict(
        llm=dict(
            provider="google", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gemini/gemini-1.5-pro-latest",
                temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google", # or openai, ollama, ...
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

In [71]:
# ✅ Configuración centralizada de parámetros
CONFIG = {
    "cliente": "MásMóvil",
    "industria": "Telecomunicaciones",
    "competidores": ["Movistar", "Vodafone", "Digi movil", "lowi", "jazztel", "Yoigo"],
    "pais": "España",
}

In [73]:
analista_industria = Agent(
    role="Analista de Inteligencia de Mercado en {industria}",
    goal="Identificar oportunidades estratégicas y tendencias emergentes en la industria para {cliente}.",
    backstory=(
        "Eres un experimentado analista con años de experiencia en investigación de mercados "
        "y tendencias en el sector de {industria}. Tu especialidad es identificar cambios en el panorama competitivo, "
        "nuevas tecnologías y factores regulatorios que impactan la industria."
    ),
    tools=[pdf_tool_industria],
    llm=deepseek_llm,
    max_iter=5
)

tarea_industria = Task(
    description=(
        "Analiza el documento de industria correspondiente a {cliente}. Extrae información sobre:\n"
        "- Factores de crecimiento y evolución del sector.\n"
        "- Impacto de regulaciones gubernamentales y tecnológicas.\n"
        "- Principales riesgos y oportunidades estratégicas.\n\n"
        "El análisis debe ser detallado, destacando insights accionables para la toma de decisiones."
    ),
    expected_output="Un informe detallado en markdown con tendencias, riesgos y oportunidades estratégicas.",
    agent=analista_industria,
    output_file="analisis_industria_{cliente}.md"
)


In [75]:
analista_satisfaccion = Agent(
    role="Especialista en Experiencia del Cliente y Retención",
    goal="Evaluar métricas de satisfacción y detectar oportunidades para mejorar la lealtad del cliente en {cliente}.",
    backstory=(
        "Como experto en experiencia del cliente, has trabajado con múltiples empresas del sector "
        "analizando patrones de satisfacción y recomendando estrategias para mejorar la retención. "
        "Tienes un enfoque basado en datos y sabes interpretar métricas clave para ofrecer recomendaciones concretas."
    ),
    tools=[pdf_tool_satisfaccion],
    llm=deepseek_llm
)

tarea_satisfaccion = Task(
    description=(
        "Realiza un análisis profundo de los datos de satisfacción del cliente en {cliente}. Identifica:\n"
        "- Puntos críticos de insatisfacción y sus causas.\n"
        "- Comparación con estándares del sector.\n"
        "- Acciones prioritarias para mejorar la experiencia del cliente.\n\n"
        "Incluye métricas clave, gráficos y conclusiones que respalden tus hallazgos."
    ),
    expected_output="Un reporte con análisis cuantitativo y cualitativo, incluyendo gráficos y recomendaciones accionables.",
    agent=analista_satisfaccion,
    output_file="reporte_satisfaccion_{cliente}.md"
)

In [1]:
analista_pricing = Agent(
    role="Especialista en Estrategia de Precios y Competencia",
    goal="Definir una estrategia de precios competitiva y rentable para {cliente}, considerando el panorama actual del mercado.",
    backstory=(
        "Eres un estratega de precios con amplia experiencia en el sector telecomunicaciones. "
        "Tu conocimiento en estructuras tarifarias, promociones y elasticidad del precio te permite identificar "
        "oportunidades para mejorar la rentabilidad y competitividad de los planes de servicio."
    ),
    tools=[search_tool, firecrawl_scrape_tool],
    llm=gemini_llm
)

tarea_pricing = Task(
    description=(
        "Realiza un análisis completo de la estrategia de precios en telecomunicaciones en {pais}, "
        "comparando a {cliente} con sus principales competidores: {competidores}.\n\n"
        "El análisis debe incluir:\n"
        "1️⃣ Descripción de los planes más representativos de cada operador (prepago, pospago, convergentes, etc.).\n"
        "2️⃣ Comparación cualitativa de estrategias de precios (¿low-cost? ¿premium? ¿diferenciación por servicio?).\n"
        "3️⃣ Identificación de ventajas competitivas de cada operador (promociones, descuentos, exclusivas, etc.).\n"
        "4️⃣ Tendencias generales del mercado y cómo afectan la estrategia de {cliente}.\n"
        "5️⃣ Sugerencias estratégicas basadas en oportunidades detectadas."
    ),
    expected_output=(
        "Un informe detallado en formato markdown con:\n"
        "✔ Un análisis por cada operador (precios, posicionamiento, estrategias clave).\n"
        "✔ Gráficos o visualizaciones de tendencias si es posible.\n"
        "✔ Recomendaciones personalizadas para {cliente}, segmentadas por tipo de usuario (ej. heavy data users, familias, low-cost seekers).\n"
    ),
    agent=analista_pricing,
    output_file="estrategia_pricing_{cliente}.md"
)


NameError: name 'Agent' is not defined

In [82]:
investigador_web = Agent(
    role="Investigador de Inteligencia Competitiva",
    goal="Descubrir estrategias de mercado de {competidores} y evaluar su impacto en {cliente}.",
    backstory=(
        "Eres un analista de mercado con experiencia en estudios de competencia en telecomunicaciones. "
        "Dominas la investigación en línea y sabes cómo identificar información clave sobre las estrategias de los competidores, "
        "incluyendo campañas de marketing, innovaciones y movimientos estratégicos."
    ),
    tools=[search_tool, firecrawl_scrape_tool],
    llm=deepseek_llm,
)

tarea_navegacion = Task(
    description=(
        "Realiza un análisis de la competencia para {cliente}, investigando las estrategias de {competidores} en {pais}.\n"
        "- ¿Qué campañas de marketing están ejecutando?\n"
        "- ¿Qué innovaciones han introducido recientemente?\n"
        "- ¿Cuáles son sus fortalezas y debilidades en comparación con {cliente}?\n\n"
        "El informe debe incluir un análisis profundo con capturas de información relevante y tendencias identificadas."
    ),
    expected_output="Un dossier detallado con información clave sobre la competencia, incluyendo estrategias y fuentes verificables.",
    agent=investigador_web,
    output_file="analisis_competitivo_{cliente}.md"
)


In [83]:
manager_agent = Agent(
    role="Coordinador de Inteligencia Estratégica",
    goal="Integrar y sintetizar todos los análisis en un informe consolidado con insights estratégicos para {cliente}.",
    backstory=(
        "Eres un ejecutivo senior con experiencia en la toma de decisiones estratégicas basadas en datos. "
        "Tu capacidad de sintetizar información compleja y traducirla en recomendaciones accionables "
        "es fundamental para definir el rumbo de {cliente} en el mercado."
    ),
    llm=gemini_llm,
    max_iter=7
)

tarea_manager = Task(
    description=(
        "Elabora un reporte ejecutivo consolidado con los hallazgos de cada análisis:\n"
        "- Resumen ejecutivo con insights clave.\n"
        "- Principales desafíos y oportunidades detectadas.\n"
        "- Recomendaciones estratégicas para mejorar la posición competitiva de {cliente}.\n\n"
        "El informe debe estar bien estructurado y listo para ser presentado a directivos."
    ),
    expected_output="Un reporte ejecutivo en markdown con insights clave, gráficos y estrategias accionables.",
    agent=manager_agent,
    output_file="reporte_final_{cliente}.md"
)


In [85]:
# 🚀 Configuración de la Crew
project_crew = Crew(
    agents=[
        analista_industria,
        analista_satisfaccion,
        analista_pricing,
        investigador_web
    ],
    tasks=[
        tarea_navegacion,
        tarea_industria,
        tarea_pricing,
        tarea_satisfaccion,
        tarea_manager  # ✅ Añadimos la tarea del manager
    ],
    manager_agent=manager_agent,
    process=Process.hierarchical,
    verbose=True,
    planning=False  # ✅ Desactivamos planificación para evitar errores
)

In [ ]:
# ⚡ Ejecución de la Crew
resultados = project_crew.kickoff(inputs=CONFIG)

# 📌 Mostrar resultados
print("\n" + "="*50)
print("Resultados del Análisis Competitivo")
print("="*50)
print(resultados)

# Agent: Coordinador de Inteligencia Estratégica
## Task: Realiza un análisis de la competencia para MásMóvil, investigando las estrategias de ['Movistar', 'Vodafone', 'Digi movil', 'lowi', 'jazztel', 'Yoigo'] en España.
- ¿Qué campañas de marketing están ejecutando?
- ¿Qué innovaciones han introducido recientemente?
- ¿Cuáles son sus fortalezas y debilidades en comparación con MásMóvil?

El informe debe incluir un análisis profundo con capturas de información relevante y tendencias identificadas.
# Agent: Investigador de Inteligencia Competitiva
## Task: Please research the following competitors of MásMóvil in Spain: Movistar, Vodafone, Digi Mobil, Lowi, Jazztel, and Yoigo. For each competitor, investigate their current marketing campaigns, recent innovations, and their strengths and weaknesses compared to MásMóvil. Please provide verifiable sources for all information.  The goal is to gather enough information to create a comprehensive competitive analysis dossier.


# Agent: Investi